# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

Data will be saved in a data folder placed in the same folder as the notebook. We check if this folder exists and if not create it

In [4]:
import os.path

dataPath = './data'
if not os.path.exists(dataPath):
    os.makedirs(dataPath)

## Question 1:
### Create a data of the postal codes, boroughs and neighbourhoods

In [6]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')

The tabular data is availabe in table and belongs to class="wikitable sortable"So let's extract only table

In [7]:
My_table = soup.find('table',{'class':'wikitable sortable'})

In [8]:
print(My_table.tr.text)


Postcode
Borough
Neighbourhood



In [9]:
headers="Postcode,Borough,Neighbourhood"

We now extract all values in tr and seperating each td within by ","

In [10]:
table1=""
for tr in My_table.find_all('tr'):
    row1=""
    for tds in tr.find_all('td'):
        row1=row1+","+tds.text
    table1=table1+row1[1:]

The extracted data will be saved in a csv file to be used to create the dataframe

In [11]:
file=open("toronto.csv","wb")
#file.write(bytes(headers,encoding="ascii",errors="ignore"))
file.write(bytes(table1,encoding="ascii",errors="ignore"))

8738

In [13]:
import pandas as pd
df = pd.read_csv('toronto.csv',header=None)
df.columns=["Postalcode","Borough","Neighbourhood"]
df.head(10)

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


Drop rows with a borough that is Not assigned

In [17]:
# Get names of indexes for which column Borough has value "Not assigned"
indexNames = df[ df['Borough'] =='Not assigned'].index

# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)
df.head(10)

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In the case of a cell having a borough but a Not assigned neighborhood, then the neighborhood will be set as the borough

In [18]:
df.loc[df['Neighbourhood'] =='Not assigned' , 'Neighbourhood'] = df['Borough']
df.head(10)

,Postalcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


Rows with the same postal code will be combined and neighbourhoods for the combined rows will be seperated by a comma

In [19]:
df_tidy = df.groupby(['Postalcode','Borough'], sort=False).agg( ', '.join)
df_tidy = df_tidy.reset_index()
df_tidy.head(15)
df_tidy.head(10)

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


The tidy dataframe will be saved into the data folder to be used in notebooks for Question 2 and 3

In [25]:
df_tidy.to_csv(dataPath + '/toronto.csv')

In [23]:
df_tidy.head()

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
